In [1]:
import pandas as pd
import copy
import requests
from tqdm.notebook import tqdm
from tqdm import tqdm
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By          # - 정보 획득
from selenium.webdriver.common.keys import Keys
# from engine_connect import create_engine_connect, db_connection
from sqlalchemy import create_engine
import os
import re
import time
from bs4 import BeautifulSoup
import warnings
import pymysql
import math
warnings.filterwarnings(action='ignore')
from selenium.webdriver.common.action_chains import ActionChains
import json
from lxml import html



import time
import datetime as dt # 사용시 datetime.dt 로 사용 # 현재 시간 사용
from datetime import datetime # 사용시 함수로 바로 사용 #-> 시간 비교

In [2]:
directory = os.getcwd()
print("Current working directory : ", directory)

#수집날짜
folder_date = str(dt.datetime.now().date())
folder_date = re.sub("-", ".", folder_date)
today = re.sub("\.", "", folder_date)
print('오늘 날짜 : ', folder_date)

now = dt.datetime.now()
print('현재 시간 : ', now)

Current working directory :  C:\Develops\summoners_war_analysis
오늘 날짜 :  2024.10.22
현재 시간 :  2024-10-22 14:14:09.085384


In [3]:
# * 웹 크롤링 동작
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

import time
# ChromeDriver 실행

from selenium.webdriver.chrome.options import Options

# Chrome 브라우저 옵션 생성
chrome_options = Options()

# User-Agent 설정
chrome_options.add_argument("--user-agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36")

# WebDriver 생성
webdriver_manager_directory = ChromeDriverManager().install()

In [4]:
# 1

# 인터넷 창 오픈

webdriver_manager_directory = ChromeDriverManager().install()
correct_webdriver_manager_directory = os.path.join(os.path.dirname(webdriver_manager_directory), "chromedriver.exe")
browser = webdriver.Chrome(service=ChromeService(executable_path=correct_webdriver_manager_directory), options=chrome_options)

# 네이버 로그인
browser.get("https://nid.naver.com/nidlogin.login")

In [6]:
# 네이버 카페 접속
browser.get("https://cafe.naver.com/smonwar")

In [7]:
# 건의사항 버튼 클릭
suggestion_category = browser.find_element(By.CSS_SELECTOR, value = "#menuLink53")
suggestion_category.click()

In [8]:
# iframe 제거하는 코드 - iframe에 해당하는 css selector 입력하면 된다.
browser.switch_to.frame("cafe_main")

In [9]:
df_suggestion = pd.DataFrame()

In [10]:
# 2

# 2020~2024년의 건의사항 크롤링

while True :
    
    page_num = browser.find_elements(By.CSS_SELECTOR, value = "div.prev-next > a:not(.pgL):not(.pgR)")
    for y in range(len(page_num)):
        page_num = browser.find_elements(By.CSS_SELECTOR, value = "div.prev-next > a:not(.pgL):not(.pgR)")
        page_num[y].click()
        time.sleep(1)
        
        # 한 페이지당 글 갯수
        length_per_page = browser.find_elements(By.CSS_SELECTOR, value = "div:nth-child(4) > table > tbody > tr")
        for x in range(len(length_per_page)):
            
            # 제목
            titles = browser.find_elements(By.CSS_SELECTOR, value = "div:nth-child(4) > table > tbody > tr > td.td_article > div.board-list > div > a.article")
            title = titles[x].text
            # 링크
            urls = titles[x].get_attribute("href")
            url = urls
            # 날짜
            dates = browser.find_elements(By.CSS_SELECTOR, value = "div:nth-child(4) > table > tbody > tr > td.td_date")
            date = dates[x].text
            # 좋아요
            likes = browser.find_elements(By.CSS_SELECTOR, value = "div:nth-child(4) > table > tbody > tr > td.td_likes")
            like = int(likes[x].text)
            
            series_addition = {"title" : [title], "url" : [url], "date" : [date], "like" : [like]}
            df_addition = pd.DataFrame(series_addition)
            df_suggestion = pd.concat([df_suggestion, df_addition], ignore_index = True)
        print(len(df_suggestion))

    # 최근 5년간 건의사항을 분석할 것이므로 2019년이 되면 break하는 코드
    year = int(date.split(".")[0])
    if year == 2019:
        break
    next_button = browser.find_element(By.CSS_SELECTOR, value = "div.prev-next > a.pgR")
    next_button.click()
    time.sleep(1.5)

15
30
45
60
75
90
105
120
135
150
165
180
195
210
225
240
255
270
285
300
315
330
345
360
375
390
405
420
435
450
465
480
495
510
525
540
555
570
585
600
615
630
645
660
675
690
705
720
735
750
765
780
795
810
825
840
855
870
885
900
915
930
945
960
975
990
1005
1020
1035
1050
1065
1080
1095
1110
1125
1140
1155
1170
1185
1200
1215
1230
1245
1260
1275
1290
1305
1320
1335
1350
1365
1380
1395
1410
1425
1440
1455
1470
1485
1500
1515
1530
1545
1560
1575
1590
1605
1620
1635
1650
1665
1680
1695
1710
1725
1740
1755
1770
1785
1800
1815
1830
1845
1860
1875
1890
1905
1920
1935
1950
1965
1980
1995
2010
2025
2040
2055
2070
2085
2100
2115
2130
2145
2160
2175
2190
2205
2220
2235
2250
2265
2280
2295
2310
2325
2340
2355
2370
2385
2400
2415
2430
2445
2460
2475
2490
2505
2520
2535
2550
2565
2580
2595
2610
2625
2640
2655
2670
2685
2700
2715
2730
2745
2760
2775
2790
2805
2820
2835
2850
2865
2880
2895
2910
2925
2940
2955
2970
2985
3000
3015
3030
3045
3060
3075
3090
3105
3120
3135
3150
3165
3180
3195
3210
32

In [11]:
df_suggestion

,title,url,date,like
0,pc 클라에서 앱을 실행할 때 주황색 알림 비활성화 옵션 건의,https://cafe.naver.com/ArticleRead.nhn?clubid=...,00:33,0
1,콜라보레이션 몬스터도 형상변환 가능하게 만들어주세요.,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2024.10.21.,0
2,2각 몬스터 스작 좀 없애주시면 안되나요?,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2024.10.21.,0
3,아레나 초기화주에 어떻게좀해봐요,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2024.10.20.,0
4,섬꾸미게 검정색 장식좀만들어줘,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2024.10.19.,0
...,...,...,...,...
6745,재미로보는 내주관적 빛암 밸패안,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2019.12.27.,0
6746,풍야만 2스 스킬로 인한 턴 오류,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2019.12.27.,0
6747,암해왕 버프 좀 해주세요,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2019.12.27.,0
6748,아니 ㅅㅂ쫌적당히좀해 확률조작이냐컴투스들아?,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2019.12.27.,0


In [12]:
df_suggestion.to_excel(f"{directory}/dataset/df_suggestion.xlsx")